In [10]:
import argparse
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
# from stage1.Network import Net
# from stage1.Dataset import FaceLandmarksDataset
# from stage1.Actions import *
from Network import Net
from Dataset import FaceLandmarksDataset
from Actions import *


DATAPATH = './'
LABLE_FILE_NAME = {'train': 'train.txt', 'test': 'test.txt'}

torch.set_default_tensor_type(torch.FloatTensor)


def main():
    # set arguments
    parser = argparse.ArgumentParser(description='Detector')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')

    parser.add_argument('--train-batch-size', type=int, default=16, metavar='N',
                       help='input batch size for training (default: 128)')
    parser.add_argument('--test-batch-size', type=int, default=16, metavar='N',
                       help='input batch size for testing (default: 128)')
    parser.add_argument('--epochs', type=int, default=1, metavar='N',
                        help='input the number of epochs to train (default: 1000)')
    parser.add_argument('--learning-rate', type=float, default=0.0001, metavar='LR',
                        help='input the learning rate of optimizer (default: 0.00008)')
    parser.add_argument('--momentum', type=float, default=0.3, metavar='M',
                        help='input the momentum of optimizer (default: 0.25)')
    parser.add_argument('--log-interval', type=int, default=20, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=True,
                        help='save the current model')
    parser.add_argument('--save-directory', type=str, default='trained_models',
                        help='the directory to save learnt models')

    parser.add_argument('--phase', type=str, default='train',
                        help='choose from Train/train, Test/test, Predict/predict, Finetune/finetune')

#     args = parser.parse_args()
    args = parser.parse_args(args=[])

    # process control
    torch.manual_seed(args.seed)
    is_use_cuda = not args.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if is_use_cuda else "cpu")
    datasets = {x: FaceLandmarksDataset(DATAPATH, LABLE_FILE_NAME[x], data_transforms[x])
                for x in ['train', 'test']}
    train_loader = DataLoader(datasets['train'], batch_size=args.train_batch_size, shuffle=True)
    test_loader = DataLoader(datasets['test'], batch_size=args.test_batch_size)
    data_loaders = {'train': train_loader, 'val': test_loader}
    model = Net().to(device)

    # training control
    criterion = nn.MSELoss(reduction='mean')
    optimizer = optim.SGD(model.parameters(), lr=args.learning_rate, momentum=args.momentum)

    # phase control
    if args.phase == 'Train' or args.phase == 'train':
        print('===> Start Training')
        lest_loss = train(args, data_loaders, model, criterion, optimizer, device)
        f = open('lest_loss_' + str(lest_loss) + '.txt', 'a+')
        [print(k, file=f) for k in args.__dict__.items()]
        [print(optimizer, file=f)]
        f.close()

    elif args.phase == 'Test' or args.phase == 'test':
        print('===> Start Testing')
        test_model = model
        test_model.load_state_dict(torch.load('./trained_models/best_model.pt'))
        avg_loss = test(test_loader, device, criterion, test_model)
        print('Average loss for test set is {:.2f}. '.format(avg_loss))

    elif args.phase == 'Finetune' or args.phase == 'finetune':
        print('===> Start Finetuning')
        pretrained_model = model
        pretrained_model.load_state_dict(torch.load('./trained_models/best_model.pt'))
        trainable_vars = [pretrained_model.ip3.bias, pretrained_model.ip3.weight]
        optimizer = optim.SGD(trainable_vars, lr=args.learning_rate, momentum=args.momentum)
        train(args, data_loaders, pretrained_model, criterion, optimizer, device)

    elif args.phase == 'Predict' or args.phase == 'predict':
        print('===> Predict')
        img_dir_name = './data/I/001063.jpg'
        test_model = model
        test_model.load_state_dict(torch.load('./trained_models/best_model.pt'))
        predict(img_dir_name, test_model, device)


if __name__ == '__main__':
    main()


===> Start Training
========== Epoch 1/1 ==========
..\data\I\001628.jpg does not exist!
..\data\I\003401.jpg does not exist!
..\data\I\000214.jpg does not exist!
..\data\II\006911.jpg does not exist!
..\data\I\004123.jpg does not exist!
..\data\I\000912.jpg does not exist!
..\data\II\009161.jpg does not exist!
..\data\II\008459.jpg does not exist!
..\data\II\008701.jpg does not exist!
..\data\I\002042.jpg does not exist!
..\data\I\004587.jpg does not exist!
..\data\I\003031.jpg does not exist!
..\data\II\008474.jpg does not exist!
..\data\II\008308.jpg does not exist!
..\data\I\005326.jpg does not exist!
..\data\II\008993.jpg does not exist!


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [8]:
import os
import cv2
import numpy as np


DATAPATH = '../data'
FOLDER_LIST = ['I', 'II']
LABLE_FILE_NAME = ['label.txt', 'label.txt']
valset_ratio = 0.3


def expand_roi(x1, y1, x2, y2, img_width, img_height, ratio):   # usually ratio = 0.25
    width = x2 - x1 + 1
    height = y2 - y1 + 1
    padding_width = int(width * ratio)
    padding_height = int(height * ratio)
    roi_x1 = x1 - padding_width
    roi_y1 = y1 - padding_height
    roi_x2 = x2 + padding_width
    roi_y2 = y2 + padding_height
    roi_x1 = 0 if roi_x1 < 0 else roi_x1
    roi_y1 = 0 if roi_y1 < 0 else roi_y1
    roi_x2 = img_width - 1 if roi_x2 >= img_width else roi_x2
    roi_y2 = img_height - 1 if roi_y2 >= img_height else roi_y2
    return roi_x1, roi_y1, roi_x2, roi_y2, \
           roi_x2 - roi_x1 + 1, roi_y2 - roi_y1 + 1


if __name__ == '__main__':
    train_set_file = open('train.txt', 'w')
    test_set_file = open('test.txt', 'w')
    for folder in range(len(FOLDER_LIST)):
        num = 1
        path = DATAPATH + '/' + FOLDER_LIST[folder]
        data_set_size = len(open(path + '/' + LABLE_FILE_NAME[folder]).readlines())
        labels = open(path + '/' + LABLE_FILE_NAME[folder])
        for line in labels:
            label = line.split()
            file_name = label[0]
            file_path = path + '/' + file_name
            x1, y1, x2, y2 = [float(i) for i in label[1:5]]
            key_points = np.array([float(i) for i in label[5:]]).reshape(-1, 2)
            try:
                img = cv2.imread(file_path)
            except Exception:
                print('Not a valid image.')
            else:
                h, w, _ = img.shape
                roi_x1, roi_y1, roi_x2, roi_y2, _, _ = expand_roi(x1, y1, x2, y2, w, h, 0.25)
                landmarks = key_points - np.array([roi_x1, roi_y1])

                #show image to check labels
                # img = img[int(roi_y1):int(roi_y2), int(roi_x1):int(roi_x2)]
                # img = cv2.rectangle(img, (int(roi_x1), int(roi_y1)), (int(roi_x2), int(roi_y2)), (0, 255, 0), 1)
                # for point in landmarks:
                #     x, y = point
                #     cv2.circle(img, (int(x), int(y)), 1, (0, 0, 255), 0)
                # cv2.imshow('img', img)
                # key = cv2.waitKey(1000)
                # if key == 27:
                #     exit(0)
                # cv2.destroyAllWindows()

                list = [str(i) for i in [roi_x1, roi_y1, roi_x2, roi_y2] + landmarks.reshape(1, -1).squeeze(0).tolist()]
                line = file_path + ' ' + ' '.join(list) + '\n'
                if num <= valset_ratio * data_set_size:
                    test_set_file.write(line)
                else:
                    train_set_file.write(line)
                num += 1
    train_set_file.close()
    test_set_file.close()
    print('Files train.txt and test.txt saved! ')


Files train.txt and test.txt saved! 
